In [65]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd
#from HEB_scraping_filter import *


In [128]:
#####################################################
################ utility function
def return_string(string, start_after_word,stop_before_word):
    """return part of string between 2 words     (string, start_after_word,stop_befor_word)"""
    try:
        if start_after_word=="":
            start_address=0
        else:
            start_address=re.search(start_after_word,string).span()[1]
        #print(start_address)
        if stop_before_word=="":
            stop_address=len(string)
        else:
            stop_address=re.search(stop_before_word,string).span()[0]
        #print(stop_address)
        my_string=string[start_address:stop_address]
    except Exception as e:
        # handle error
        error=f"STRING _exception_: {type(e).__name__},</br> _arguments_: {e.args}"
        #print(error)
        my_string=None
    return my_string


# In[7]:


####################################################
###############filter function 
def heb_filter(soup_find):
    """ function takes result of HEB scraping - soup.find_all and
    transforms it to dataframe """
    my_result=soup_find
    scrap_df=pd.DataFrame()
#   my_result=soup.body.find_all('a',id=re.compile('product-'))
       
    i=0
    for record in my_result:
        json=eval(record.find('script',type="application/ld+json").string)
        try:
            try:
                name=record.find('span',class_=re.compile('responsive')).text.strip()
            except:
                name=json['name']
            size=return_string(name,"Milk","")
            json=eval(record.find('script',type="application/ld+json").string)
            id=json['id']
            brand=json['brand']
            category=json['category']
            price=json['price']
            search_string=record['aria-label'].strip()
            features=return_string(search_string,"Features:","")
         #  rating=return_string(search_string,"Rated","stars")
            try:
                uomSalePrice=record.find('span',class_=re.compile('uomSalePrice')).text.strip()
            except:
                uomSalePrice=None
            try:
                image=record.find('img',attrs={"data-src":re.compile('prd-small')})['data-src']
            except:
                image=record.find('img',attrs={"src":re.compile('prd-small')})['src']
            if record.find('img',src=re.compile('coupon'))==None:
                coupon=0
            else:
                coupon=1
            #debug prints
            #print(i) 
            #print (f'{name} \n{size}\n{json}\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')
            #print('___________________')
            i+=1
            scrap_df = scrap_df.append({'id':id,
                                'name': name,
                                'brand':brand,
                                'size':size,
                                'category':category,
                                'price':price,
                                'features':features,
                                'coupon':coupon,
                                'uomSalePrice':uomSalePrice,
                                'image':image,
                               }, ignore_index=True)
        except Exception as e:
        # handle error
            print(i) 
            print (f'{name} \n{size}\n{json}\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')
            error=f"MAIN_exception_: {type(e).__name__},</br> _arguments_: {e.args}"
            print(error)
        
    return scrap_df

In [28]:
#setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\ykras\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [40]:
milk = "https://www.heb.com/search/results?N=3139289889&No=0&Nrpp=60&Ntt=milk&q=milk&prodFilter=none"
browser.visit(milk)
time.sleep(10)

In [42]:
html = browser.html
soup = bs(html,"lxml")

page = soup.body.find_all('a',id=re.compile('product-'))
milks = page
browser.links.find_by_partial_text('Next').click()
time.sleep(10)
for x in range(1,15):
    html = browser.html
    soup = bs(html,"lxml")
    page = soup.body.find_all('a',id=re.compile('product-'))
    milks.extend(page)
    try:
        browser.links.find_by_partial_text('Next').click()
        time.sleep(10)
    except:
        break

In [140]:
milks[675]
# tag_string = ""
# for item in milks[0].contents[1]:
#     tag_string += str(item)
    
# #tag_string
# item_info = tag_string.split("{")
# the_right_stuff = item_info[1]
# while "\t" in the_right_stuff:
#     index = the_right_stuff.find('\t')
#     the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+1:]
# while "\n" in the_right_stuff:
#     index = the_right_stuff.find('\n')
#     the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+1:]
# while "</script>" in the_right_stuff:
#     index = the_right_stuff.find('</script>')
#     the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+9:]
# while "</div>" in the_right_stuff:
#     index = the_right_stuff.find('</div>')
#     the_right_stuff = the_right_stuff[:index] + the_right_stuff[index+6:]
# the_right_stuff

<a aria-label="H-E-B Baby Toddler Beginnings Stage 2 Formula with (9-18 Months),
		22 oz,
		$12.24 each.Coupon available." class="responsivegriditem__aria-link-wrap" href="/product-detail/h-e-b-baby-toddler-beginnings-stage-2-formula-with-9-18-months-/1412004" id="product-1412004">
<div class="responsivegriditem-top is-carousel" onclick="gtm_e_productClick('gtm-product-click-1412004');">
<div class="cat-feature clearfix">
<div class="cat-feature-marketing">
</div>
<div class="cat-feature-list clearfix">
</div></div>
<div class="cat-list-img gtm-product gtm-carousel">
<img alt="H-E-B Baby Toddler Beginnings Stage 2 Formula with (9-18 Months)" class="ls-is-cached lazyloaded" src="https://images.heb.com/is/image/HEBGrocery/prd-small/h-e-b-baby-toddler-beginnings-stage-2-formula-with-9-18-months--001412004.jpg"/><script type="application/ld+json">
{	
	"name": "H-E-B Baby Toddler Beginnings Stage 2 Formula with (9-18 Months)", 
	"id": "1412004", 
	"category": "Baby & Kids/Food & Formula/For

In [44]:
browser.quit()

In [129]:
record=milks[660]
record.find('img',attrs={"src":re.compile('prd-small')})['src']

'https://images.heb.com/is/image/HEBGrocery/prd-small/similac-go-grow-non-gmo-toddler-drink-powder-002070468.jpg'

In [126]:
json=eval(record.find('script',type="application/ld+json").string)
#name=record.find('span',class_=re.compile('responsive')).text.strip()
name=json['name']
size=return_string(name,"Milk","")

id=json['id']
brand=json['brand']
category=json['category']
price=json['price']
search_string=record['aria-label'].strip()
features=return_string(search_string,"Features:","")
# #  rating=return_string(search_string,"Rated","stars")
uomSalePrice=record.find('span',class_=re.compile('uomSalePrice')).text.strip()
#image=record.find('img',attrs={"data-src":re.compile('prd-small')})['data-src']
print (f'{name}\n{size}\n{json}\n{id}\n{brand}\n{category}\n{price}')
#\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')

Similac Go & Grow Non-GMO Toddler Drink Powder
None
{'name': 'Similac Go & Grow Non-GMO Toddler Drink Powder', 'id': '2070468', 'category': 'Baby & Kids/Food & Formula/Formula', 'brand': 'Similac', 'variant': 'each', 'price': 20.58, 'list': 'We recommend', 'position': '1', 'dimension26': 'false', 'dimension40': 'default'}
2070468
Similac
Baby & Kids/Food & Formula/Formula
20.58


In [130]:
scrap_df=heb_filter(milks)

In [142]:
heb_df=scrap_df.loc[scrap_df['category']=='Dairy & Eggs/Milk']

In [144]:
heb_df.loc[heb_df['name']=='Thai Kitchen Organic Unsweetened Coconut Milk, 13.66 oz']

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
97,Thai Kitchen,Dairy & Eggs/Milk,0.0,LESS THAN 5G NET CARBS. VEGETARIAN. VEGAN....,740626,https://images.heb.com/is/image/HEBGrocery/prd...,"Thai Kitchen Organic Unsweetened Coconut Milk,...",2.46,", 13.66 oz",($0.19/oz)


In [14]:
path='data/heb_scrape.csv'
scrap_df.to_csv(path, index_label=False)